In [20]:
import pandas as pd
import smartsheet
import openpyxl
import numpy as np
import os
import docx
import docx2pdf
from docx2pdf import convert
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
from docx.enum.table import WD_ALIGN_VERTICAL
from docx.enum.text import WD_BREAK
from docx.shared import Pt
from smartsheet_dataframe import get_as_df, get_report_as_df, get_sheet_as_df
from datetime import datetime
import jdcal
import re


# using API Key to get nesscessary report/sheet

api_key = 'insert_your_key'
smartsheet_client = smartsheet.Smartsheet(api_key)
# co = smartsheet_client.sheet, get_sheet_as_df(sheet_id)

In [21]:
sheet = smartsheet_client.Sheets.get_sheet(sheet_id=sheet_id)


In [22]:
co_df = get_sheet_as_df(token=api_key, sheet_id=sheet_id)
co_df = co_df.iloc[:, 3:]
# co_df

# PENDING CHANGE ORDERS

In [23]:
need_co = co_df.loc[co_df['CO Created'] == 'No']
need_co = need_co[['Program', 'Ward', 'Region', 'WO#', 'Address', 'Description', 'Release #', 'Hours', 'Calculated Amount']]
# need_co

In [24]:
w12 = need_co.loc[(need_co['Program']=='New Business') & (need_co['Region']=='DC') & (need_co['Ward']=='Wards 1&2')]
w34 = need_co.loc[(need_co['Program']=='New Business') & (need_co['Region']=='DC') & (need_co['Ward']=='Wards 3&4')]
w56 = need_co.loc[(need_co['Program']=='New Business') & (need_co['Region']=='DC') & (need_co['Ward']=='Wards 5&6')]
w78 = need_co.loc[(need_co['Program']=='New Business') & (need_co['Region']=='DC') & (need_co['Ward']=='Wards 7&8')]
pgn = need_co.loc[(need_co['Program']=='New Business') & (need_co['Region']=='MD') & (need_co['Ward']=='PG North')]
pgs = need_co.loc[(need_co['Program']=='New Business') & (need_co['Region']=='MD') & (need_co['Ward']=='PG South')]
pl = need_co.loc[(need_co['Program']=='New Business') & (need_co['Region']=='MD') & (need_co['Ward']=='Purple Line')]

hw_rd = need_co.loc[(need_co['Program']=='Howard Road') & (need_co['Region']=='DC') & (need_co['Ward']=='Wards 7&8')]

nem = need_co.loc[(need_co['Program']=='NEM')]
nemdc = need_co.loc[(need_co['Program']=='NEM') & (need_co['Region']=='DC') & (need_co['Ward']=='None')]
nemmd = need_co.loc[(need_co['Program']=='NEM') & (need_co['Region']=='MD') & (need_co['Ward']=='None')]
dtpa = need_co.loc[(need_co['Program']=='DTPA')]
pb = need_co.loc[(need_co['Program']=='Pipe Bus')]

co_groups = [w12, w34, w56, w78, pgn, pgs, pl, hw_rd,  nemdc, nemmd, dtpa, pb]


In [25]:
for prgm in co_groups:
    print(len(prgm))


6
0
0
0
0
0
0
0
0
0
0
0


In [ ]:
excel_path = r'WAF MASTER.xlsx'

waf_db = pd.read_excel(excel_path, sheet_name='WAF DATABASE')

waf_db = waf_db.iloc[:, :12]
# waf_db

In [27]:
# code for investigation when needed


# testwafdb = waf_db.loc[(waf_db['PROGRAM AND AREA'] == 'NB DC Wards 3&4')]
# testwafdb['DOLLAR AMOUNT'].values

In [28]:
# waf_db.loc[(waf_db['WO'] == 19380518) & (waf_db['PROGRAM AND AREA'] == 'NB DC Wards 3&4'), 'DOLLAR AMOUNT'].values[0]

# og_amount = float(tempwafdb['DOLLAR AMOUNT'].values[0])

In [29]:
# tempwafdb = waf_db.loc[(waf_db['WO'] == 19173621) & (waf_db['PROGRAM AND AREA'] == 'NB DC Wards 7&8')]

# og_amount = float(tempwafdb['DOLLAR AMOUNT'].values[0])

# og_amount + 0


# waf_db.loc[(waf_db['WO'] == 19680108) & (waf_db['PROGRAM NAME'] == 'New Business MD')]
# raw_df2.loc[(raw_df2['Managers'] != 'PPE') & (raw_df2['Managers'] == 'Tamrat Abegaz')]

In [31]:
# Print COORDINATES when needed for Table in CO Document for a clearer view

# for i, row in enumerate(doc.tables[0].rows):
#     for j, cell in enumerate(row.cells):
#         print(i, j)

#         for paragraph in cell.paragraphs:
#             print(paragraph.text)

        
#         print("-" * 50)

# PROGRAM DICTIONARY

In [32]:
program_map = {
    'DTPA': 'DTPA',
    'DTPA DC': 'DTPA',
    'DTPA MD': 'DTPA',
    # 'Job Release DC': 'New Business DC',
    # 'Job Release MD': 'New Business MD', # Make sure Job Release and Tech QAQC have regions filled out MIGHT NOT EVEN NEED THIS ANYMORE
    # 'Technical QA/QC DC': 'New Business DC',
    # 'Technical QA/QC MD': 'New Business MD',
    'Pipe Bus': 'Pipe Bus',
    'New Business DC': 'New Business DC',
    'New Business MD': 'New Business MD',
    'Howard Road': 'New Business DC',
    'NEM DC': 'NEM',
    'DTPA ': 'DTPA',
    'NEM MD': 'NEM',
    "DTPA  ":'DTPA', 
    "Howard Road DC" : "Howard Road",
    "Pipe Bus DC" : "Pipe Bus"
}

nb_stuf = {
    'DTPA': 'DTPA',
    'New Business': 'NB',
    'New Business': 'NB',
    "Howard Road DC" : "Howard Road",
    "Howard Road" : "Howard Road",
    "Pipe Bus" : "Pipe Bus"
}

In [33]:
def Change_Order_Mapper(Program, ReleaseNum, Ward, WONum, Location, Description, Region, Hours, CalculatedCost, OrigCost, RevisedCost, MasterList, WOText, LocationText):
    
    #loading in template
    # template_doc = docx.Document("Change Order Template2.docx")
    template_doc = docx.Document("Change Order Template3.docx")
    # print(ReleaseNum[0])

    #replacing cell text with CO text
    template_doc.tables[0].cell(3, 0).text = str(WOText) #'See Table Below' #str(WONum)
    template_doc.tables[0].cell(1, 1).text = '00' + str(int(ReleaseNum[0]))
    template_doc.tables[0].cell(3, 2).text = str(LocationText) #'See Table Below' #str(Location)
    template_doc.tables[0].cell(3, 3).text = Region[0] #str(Region)
    template_doc.tables[0].cell(5, 0).text = 'See Table Below' #str(Description)
    template_doc.tables[0].cell(2, 4).text = '${:.2f}'.format(sum(CalculatedCost)) # add coordinates for og and revised costs
    template_doc.tables[0].cell(8, 5).text = '${:.2f}'.format(sum(CalculatedCost))  # edit indentations on word document    
    template_doc.tables[0].cell(10, 5).text = '${:.2f}'.format(sum(OrigCost))
    template_doc.tables[0].cell(11, 5).text = '${:.2f}'.format(sum(RevisedCost))
    template_doc.tables[0].cell(17, 8).text = '${:.2f}'.format(sum(CalculatedCost))
    
    if len(Description) == 1:
        Description = Description[0]  # If there's only one value, return it without any delimiter
        # print(descr_list)

    if isinstance(Description, list):
            template_doc.tables[0].cell(5, 0).text = 'See Table Below' #str(Description)
    elif isinstance(Description, str):
             template_doc.tables[0].cell(5, 0).text = str(Description)

    # workorder = WONum
    # addy = Location
    # reason = Description
    # hrs = Hours

    # formatting
    coordinates = [(8, 5), (10, 5), (11, 5)]
    for row, column in coordinates:
        # Access the specific cell using the coordinates
        cell = template_doc.tables[0].cell(row, column)

        # Iterate through paragraphs in the cell and modify indentation
        for paragraph in cell.paragraphs:
            # paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.LEFT
            paragraph.style.paragraph_format.left_indent = int(round((0.0 * 914400), 0))  # Converting inches to Twips (1 inch = 914400 Twips)
            # paragraph.style.paragraph_format.right_indent = 0 * 914400  

    coords = [(3, 0), (1, 1), (3, 2), (3, 3), (2, 4)]
    for row, column in coords:
        # Access the specific cell using the coordinates
        cell = template_doc.tables[0].cell(row, column)

        # Iterate through paragraphs in the cell and modify indentation
        for paragraph in cell.paragraphs:
            paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
            paragraph.style.paragraph_format.left_indent = int(round((0.0 * 914400), 0))

    template_doc.add_paragraph()
    template_doc.add_paragraph()


    header_values = ['WO#', 'Location', 'Description', 'Hours', 'Calculated Cost', 'Original Cost', 'Revised Cost']
    max_length = len(MasterList[0])
    table = template_doc.add_table(rows=max_length + 1, cols= len(header_values))
    table.style = 'Table Grid'
    
    # Add header row with designated text values
    for j, header_value in enumerate(header_values):
        cell = table.cell(0, j)
        cell.text = header_value

    # Add data to the table starting from second row
    max_length = len(MasterList[0])
    for j in range(len(header_values)):
        for i in range(len(MasterList[0])):
                cell = table.cell(i + 1, j)
                cell.text = str(MasterList[j][i])


    # adjusting table properties 
    table.autofit = True  
    for row in table.rows:
        for cell in row.cells:
            cell.vertical_alignment = WD_ALIGN_VERTICAL.CENTER  
            cell.width = Pt(100)  

    
    for row in table.rows:
        for cell in row.cells:
            paragraphs = cell.paragraphs
            for paragraph in paragraphs:
                for run in paragraph.runs:
                    run.font.size = Pt(10)
    

    current_date = datetime.today()

    jdn = sum(jdcal.gcal2jd(current_date.year, current_date.month, current_date.day)) # julian date number

    jdn = (int(round(jdn, 0)))
    date_string = current_date.strftime("%m/%d/%Y")

    # print(date_string)
    template_doc.tables[0].cell(26, 1).text = date_string # updating the Date Field


    print(Program[0].strip(), Ward[0])
    
    if re.search(r"\bDTPA\b", Program[0], re.IGNORECASE):
        template_doc.save(f"C:\\Users\\TemitopeOmobo\\Documents\Change Orders\\CO Automation\\{Program[0].strip()}\\{nb_stuf[Program[0]].strip()} Change Order {jdn}.docx")
        convert(f"C:\\Users\\TemitopeOmobo\\Documents\Change Orders\\CO Automation\\{Program[0]}\\{nb_stuf[Program[0]]} Change Order {jdn}.docx")
    else:
        template_doc.save(f"C:\\Users\\TemitopeOmobo\\Documents\Change Orders\\CO Automation\\{(Program[0].strip() + ' ' + Region[0].strip() + ' ' + Ward[0].strip()).strip()}\\{nb_stuf[Program[0]].strip() + ' ' + Region[0].strip() + ' ' + Ward[0].strip()} Change Order {jdn}.docx")
        convert(f"C:\\Users\\TemitopeOmobo\\Documents\Change Orders\\CO Automation\\{Program[0] + ' ' + Region[0] + ' ' + Ward[0].strip()}\\{nb_stuf[Program[0]].strip() + ' ' + Region[0].strip() + ' ' + Ward[0].strip()} Change Order {jdn}.docx")


        return MasterList

    

In [34]:
def Create_Change_Order(Dataframe):
    prg_list = []
    ward_list = []
    wo_list = []
    addy_list = []
    descr_list = []
    rele_list = []
    rgn_list = []
    hrs_list = []
    calc_list = []
    orig_num_list = []
    revised_num_list = []
    pristine_calc = []
    pristine_orig = []
    pristine_revised = []


    for i, row in Dataframe.iterrows():
        
        if row['Ward'] == '' or row['Ward'] == "None":
            prg_area = str(row['Program'].strip())
        elif (row['Program'] == 'New Business') and ((row['Region'] == 'MD') or (row['Region'] == 'DC')):
            prg_area = str(row['Program'].strip()) + ' ' + str(row['Region'].strip())
        else:
            prg_area = str(row['Program'].strip()) + ' ' + str(row['Region'].strip()).strip()

        prg_list.append(row['Program'].strip())
        ward_list.append(row['Ward'].strip())
        wo_list.append(int(row['WO#']))
        addy_list.append(row['Address'].strip())
        descr_list.append(row['Description'].strip())
        rele_list.append(row['Release #'])
        rgn_list.append(row['Region'].strip())
        hrs_list.append(row['Hours'])
        calc_list.append((row['Calculated Amount']))

        print(row['WO#'], program_map[prg_area])
        orig_num_list.append(waf_db.loc[(waf_db['WO'] == row['WO#']) & (waf_db['PROGRAM NAME'] == program_map[prg_area]), 'DOLLAR AMOUNT'].values[0])
        revised_num_list.append(waf_db.loc[(waf_db['WO'] == row['WO#']) & (waf_db['PROGRAM NAME'] == program_map[prg_area]), 'DOLLAR AMOUNT'].values[0] + row['Calculated Amount'])
        pristine_calc.append('${:.2f}'.format(row['Calculated Amount']))
        pristine_orig.append('${:.2f}'.format(waf_db.loc[(waf_db['WO'] == row['WO#']) & (waf_db['PROGRAM NAME'] == program_map[prg_area]), 'DOLLAR AMOUNT'].values[0]))
        pristine_revised.append('${:.2f}'.format(waf_db.loc[(waf_db['WO'] == row['WO#']) & (waf_db['PROGRAM NAME'] == program_map[prg_area]), 'DOLLAR AMOUNT'].values[0] + row['Calculated Amount']))

    if len(wo_list) == 1:
        wo_list_text = wo_list[0]  # If there's only one value, return it without any delimiter
    else:
        wo_list_text = ""
        for value in wo_list:
            wo_list_text += f"{value}"  
            wo_list_text += " | "
        # Remove the last " | " delimiter
        wo_list_text = wo_list_text[:-3]

    if len(addy_list) == 1:
        addy_list_text = addy_list[0]  # If there's only one value, return it without any delimiter
    else:
        addy_list_text = ""
        for value in addy_list:
            addy_list_text += f"{value}"  
            addy_list_text += " | "
        # Remove the last " | " delimiter
        addy_list_text = addy_list_text[:-3]

    Release_Number = '00' + str(int(rele_list[0]))
    data = [wo_list, addy_list, descr_list, hrs_list, pristine_calc, pristine_orig, pristine_revised] #COULD add Region back in
    Change_Order_Mapper(prg_list, rele_list, ward_list, wo_list, addy_list, descr_list, rgn_list, hrs_list, calc_list, orig_num_list, revised_num_list, data, wo_list_text, addy_list_text)


    

# Create CO

In [35]:
import time
try:
    for prgm in co_groups:
        if len(prgm) > 0:
            data = Create_Change_Order(prgm[['Program', 'Ward', 'Region', 'WO#', 'Address', 'Description', 'Release #', 'Hours', 'Calculated Amount']])
            time.sleep(5) # Adding buffer to prevent errors from code not closing word doc fast enough
except IndexError:
     print('WO NOT FOUND IN WAF DB \nMost likely')




18520532.0 New Business DC
18296845.0 New Business DC
19789591.0 New Business DC
19789592.0 New Business DC
20105558.0 New Business DC
19646451.0 New Business DC
New Business Wards 1&2


100%|██████████| 1/1 [00:17<00:00, 17.50s/it]


In [38]:


# current_date = datetime.today()

# jdn = sum(jdcal.gcal2jd(current_date.year, current_date.month, current_date.day))

# date_string = current_date.strftime("%m/%d/%Y")

# print(date_string)

# print(int(round(jdn, 0)))